# Conditional Formatting

*New in version 0.17.1*

<p style="color: red">*Provisional: This is a new feature and still under development. We'll be adding features and possibly making breaking changes in future release. We'd love to hear your [feedback](https://github.com/pydata/pandas/issues).*<p style="color: red">

You can apply **conditional formatting**, the visual styling of a DataFrame
depending on the data within, by using the ``DataFrame.style`` property.
This is a property that returns a ``pandas.Styler`` object, which has
useful methods for formatting and displaying DataFrames.

The styling is accomplished using CSS.
You write functions that take scalars, `DataFrame`s or `Series`, and return *like-indexed* DataFrames or Series with CSS `"attribute: value"` pairs for the values.
These functions can be incrementally passed to the `Styler` which collects the styles before rendering.

# Building Styles

Pass your style functions into one of the following methods:

- `Styler.applymap`: elementwise
- `Styler.apply`: column/row-wise/tablewise

Each of those methods take a function (and some other keyword arguments) and apply your function to the DataFrame in a certain way. `applymap` works through the DataFrame elementwise, `apply` passes each column or row into your DataFrame one-at-a-time or the entire table at once, depending on the `axis` keyword argument. For columnwise use `axis=0`, rowwise use `axis=1`, and for the entire table at once use `axis=None`.

The result of the function application, a CSS attribute-value pair, is stored in an internal dictionary on your ``Styler`` object.

Let's create a DataFrame to work with.

In [30]:
import pandas as pd
import numpy as np

np.random.seed(24)
df = pd.DataFrame({'A': np.linspace(1, 10, 10)})
df = pd.concat([df, pd.DataFrame(np.random.randn(10, 4), columns=list('BCDE'))],
               axis=1)
df.iloc[0, 2] = np.nan

Here's a (boring) example of rendering a DataFrame, without any visible styles:

In [32]:
df.style

*Note*: The `DataFrame.style` is a propetry that returns a `Styler` object. `Styler` has a `_repr_html_` method defined on it so they are rendered automatically. If you want the actual HTML back for further processing or for writing to file call the `.render()` method which returns a string.

The above output looks very similar to the standard DataFrame HTML representation. But we've done some work behind the scenes to attach CSS classes to each cell. We can view these by calling the `.render` method.

In [3]:
df.style.render(trim=False).split('\n')[:10]  # ignore trim for now

['',
 '            <style  type="text/css" >',
 '            ',
 '            ',
 '                #T_9a2da04a_8644_11e5_9f81_a45e60bd97fbrow0_col0 {',
 '                ',
 '                }',
 '            ',
 '                #T_9a2da04a_8644_11e5_9f81_a45e60bd97fbrow0_col1 {',
 '                ']

The `row0_col0` is the identifier for that particular cell. We've also prepended each row/column identifier with a UUID unique to each DataFrame so that the style from one doesn't collied with the styling from another within the same notebook or page (you can set the `uuid` if you'd like to tie together the styling of two DataFrames).

Let's write a simple style function that will color negative numbers red and positive numbers black.

In [33]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color

In this case, the cell's style depends only on it's own value.
That means we should use the `Styler.applymap` method which works elementwise.

In [34]:
df.style.applymap(color_negative_red)

Notice the similarity with the standard `df.applymap`, which operates on DataFrames elementwise. We want you to be able to resuse your existing knowledge of how to interact with DataFrames.

Notice also that our function returned a string containing the CSS attribute and value, separated by a colon just like in a `<style>` tag. This will be a common theme.

Now suppose you wanted to highlight the maximum value in each column.
We can't use `.applymap` anymore since that operated elementwise.
Instead, we'll turn to `.apply` which operates columnwise (or rowwise using the `axis` keyword). Later on we'll see that something like `highlight_max` is already defined on `Styler` so you wouldn't need to write this yourself.

In [35]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

In [7]:
df.style.apply(highlight_max)

We encourage you to use method chains to build up a style piecewise, before finally rending at the end of the chain.

In [8]:
df.style.\
    applymap(color_negative_red).\
    apply(highlight_max)

Above we used `.apply` to pass in each column one at a time.
What if you wanted to highlight just the maximum value in the entire table?
Use `.apply(function, axis=None)` to indicate that your function wants the entire table, not one column or row at a time.

Let's rewrite our `highlight-max` to handle either Series (from `.apply(axis=0)`) or DataFrames (from `.apply(axis=1)`). We'll also allow the color to be adjustable, to demonstrate that `.apply`, and `.applymap` pass along keyword arguments.

In [9]:
def highlight_max(data, color='yellow'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

In [10]:
df.style.apply(highlight_max, color='darkorange', axis=None)

You can control the precision of floats using pandas' regular `display.precision` option.

In [11]:
with pd.option_context('display.precision', 2):
    html = (df.style
              .applymap(color_negative_red)
              .apply(highlight_max))
html

## Finer Control: Slicing

The signatures for `Styler.apply`, `Styler.applymap` and `Styler.tee` all include a `subset` keyword.
This allows you to apply styles to specific rows or columns, without having to code that logic into your `style` function.

The value passed to `subset` behaves simlar to slicing a DataFrame. You can pass a list (or Series or numpy array) of column lables to select just those columns.

In [12]:
df.style.apply(highlight_max, subset=['B', 'C', 'D'])

For row and column slicing, any valid indexer to `.loc` will work.

In [13]:
df.style.applymap(color_negative_red,
                  subset=pd.IndexSlice[2:5, ['B', 'D']])

Consider using `pd.IndexSlice` to make writing the slices easier.

**N.B.** Only label-based slicing is supported right now, not positional.

**N.B.** If your style function uses a `subset` keyword argument, consider wrapping your function in a `functools.partial`, partialing out that keyword.

```python
my_func2 = functools.partial(my_func, subset=42)
```

## Builtins

Finally, we expect certain styling functions to be common enough that we've included a few "built-in" to the `Styler`, so you don't have to write them yourself.

In [14]:
df.style.highlight_null(null_color='red')

Gradients for the background color. These require matplotlib, and we'll use [Seaborn](http://stanford.edu/~mwaskom/software/seaborn/) to get a nice colormap.

In [15]:
import seaborn as sns

cm = sns.light_palette("green", as_cmap=True)

s = df.style.background_gradient(cmap=cm)
s

`Styler.background_gradient` takes the keyword arguments `low` and `high`. Roughly speaking these extend the range of your data by `low` and `high` percent so that when we convert the colors, the colormap's entire range isn't used. This is useful so that you can actually read the text still.

In [19]:
# Uses the full color range
df.loc[:4].style.background_gradient(cmap='viridis')

In [20]:
# Compreesses
(df.loc[:4]
    .style
    .background_gradient(cmap='viridis', low=.5, high=0)
    .highlight_null('red'))

You got your barchart in my DataFrame!

In [21]:
df.style.bar(subset=['A', 'B'], color='#7F7FFF')

Use `Styler.set_properties` when the style doesn't actually depend on the values.

In [22]:
df.style.set_properties(**{'background-color': 'black',
                           'color': 'lawngreen',
                           'border-color': 'white'})

## Other options

You've seen a few methods for data-driven styling.
`Styler` also provides a few other options for styling that don't depend on the data.

- captions
- table-wide styles

Each of these can be specified in two ways:

- A keyword argument to `pd.Styler`
- A keyword argument to `.render`


### Captions

In [23]:
from pandas.core.style import Styler
s = Styler(df, caption='Colormaps, with a caption.')
s.background_gradient(cmap=cm)

Alternativly, pass the caption in when rendering.

In [24]:
from IPython.display import HTML
HTML(
    df.style.background_gradient(cmap=cm).render(caption="Caption, from render.")
)

### Table Styles

The next option you have are "table styles".
These are styles that apply to the table as a whole, and don't look at the data.
Certain sytlings, including pseudo-selectors like `:hover` can only be used this way.

In [25]:
def hover(hover_color="#ffff99"):
    return dict(selector="tr:hover", props=[("background-color", "%s" % hover_color)])

styles = [hover(), dict(selector="th", props=[("font-size", "150%"),
                                              ("text-align", "center")])]
html = df.style.render(table_styles=styles, caption="Hover to highlight.")
HTML(html)

Fun stuff.

In [26]:
# https://developer.mozilla.org/en-US/docs/Web/CSS/animation#Cylon_Eye
# no animation yet :(

def cylon(s):
    tpl = """
  background-color: red;
  background-image: -webkit-linear-gradient(    left, rgba( 0,0,0,0.9 ) 25%, rgba( 0,0,0,0.1 ) 50%, rgba( 0,0,0,0.9 ) 75%);
  background-image:    -moz-linear-gradient(    left, rgba( 0,0,0,0.9 ) 25%, rgba( 0,0,0,0.1 ) 50%, rgba( 0,0,0,0.9 ) 75%);
  background-image:      -o-linear-gradient(    left, rgba( 0,0,0,0.9 ) 25%, rgba( 0,0,0,0.1 ) 50%, rgba( 0,0,0,0.9 ) 75%);
  background-image:         linear-gradient(to right, rgba( 0,0,0,0.9 ) 25%, rgba( 0,0,0,0.1 ) 50%, rgba( 0,0,0,0.9 ) 75%);
  color: white;
  height: 100%;
  width: 20"""
    return pd.Series([tpl for i in s], index=s.index, name=s.name)

In [27]:
df.style.apply(cylon)

Interacts pretty well with widgets. If you're viewing this on NBViewer you're missing out on some interactivity.

In [28]:
from IPython.html import widgets
@widgets.interact
def f(h_neg=(0, 359, 1), h_pos=(0, 359), s=(0., 99.9), l=(0., 99.9)):
    return df.style.background_gradient(
        cmap=sns.palettes.diverging_palette(h_neg=h_neg, h_pos=h_pos, s=s, l=l,
                                            as_cmap=True)
    )

In [29]:
# Don't do this
def panda(df):
    spots = [
        # left ear
        (0, 0), (1, 0), (0, 1),
        # right ear
        (0, 10), (1, 10), (0, 9),
        # left eye
        (5, 2), (4, 3), (5, 3),
        # right eye
        (5, 7), (4, 7), (5, 8),
        # nose
        (10, 4), (10, 5), (10, 6), (11, 5),
        (15, 5), (16, 5), (17, 5), (17, 4), (17, 6), (17, 3), (17, 7), (16, 2), (16, 8)
    ]

    rows = []
    for r in df.index:
        cols = []
        for c in df.columns:
            if (r, c) in spots:
                cols.append("background-color: black; color: white")
            else:
                cols.append("background-color: white; color: black")
        rows.append(cols)

    return pd.DataFrame(rows)

df = pd.DataFrame(np.random.randn(20, 11))
s = df.round(2).style
s._update_ctx(panda(df))
s

# Subclassing

Unlike most objects that pandas exposes, `Styler` should be designed with subclassing in mind. There's no particular reason why CSS should be used over other backend, other than it's convinient for users of Notebooks, and relatively straightforward compared to, say, $\LaTeX$. I've laid out a bit of the internal implementation here.

As users apply styles (via `.apply`, `.applymap` and `.tee`), we modify an internal dict, `self.ctx`. This maps `(row_position, col_position)` to a list of (for CSS) `'attribute: value:` strings. Given this `ctx`, the rendering roughly consists of two steps

- `translate` to a dictionary ready to be passed into the template `Styler.t`
- render template with the dictionary from `Styler.translate`

This is done in `Styler.render`, but most of the heavy lifitng is done in `translate`.


## Alternate templates

We've used [Jinja](http://jinja.pocoo.org/) templates to build up the HTML.
The template is stored as a class variable on `Styler` (called `Styler.t` for now) Subclasses can override that.

```python
class CustomStyle(Styler):
    t = Template("""...""")
```